In [ ]:
#default_exp model.model

In [ ]:
#export
import torch
import torch.nn as nn
import transformers

In [ ]:
#export
class EntityModel(nn.Module):
    def __init__(self, num_tag, num_pos, drop=0.3, model_name='bert-base-uncased'):
        super().__init__()
        _config = transformers.BertConfig.from_pretrained(model_name)
        self.model = transformers.BertModel.from_pretrained(model_name)#, config=_config)
        
        self.num_tag = num_tag
        self.num_pos = num_pos
        
        out_size = getattr(self.model, 'pooler').dense.out_features
        self.drop_tag = nn.Dropout(drop)
        self.drop_pos = nn.Dropout(drop)

        self.classifier_tag = nn.Linear(out_size, self.num_tag)
        self.classifier_pos = nn.Linear(out_size, self.num_pos)

    def forward(self, input_ids, attention_mask, token_type_ids):
        enc_hidden, _ = self.model(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                           )
#         enc_hidden: (batch_size, sequence_length, hidden_size)
        out_tag = self.drop_tag(enc_hidden)
        out_pos = self.drop_pos(enc_hidden)
        
        return self.classifier_tag(out_tag), self.classifier_pos(out_pos)

In [ ]:
#export
class EntityModelWithLoss(nn.Module):
    def __init__(self, num_tag, num_pos, drop=0.3, model_name='bert-base-uncased'):
        super().__init__()
        _config = transformers.BertConfig.from_pretrained(model_name)
        self.model = transformers.BertModel.from_pretrained(model_name)#, config=_config)
        
        self.num_tag = num_tag
        self.num_pos = num_pos
        
        out_size = getattr(self.model, 'pooler').dense.out_features
        self.drop_tag = nn.Dropout(drop)
        self.drop_pos = nn.Dropout(drop)

        self.classifier_tag = nn.Linear(out_size, self.num_tag)
        self.classifier_pos = nn.Linear(out_size, self.num_pos)

    def forward(self, input_ids, attention_mask, token_type_ids, target_tag, target_pos):
        enc_hidden, _ = self.model(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                           )
#         enc_hidden: (batch_size, sequence_length, hidden_size)
        out_tag = self.drop_tag(enc_hidden)
        out_pos = self.drop_pos(enc_hidden)
        
        out_tag = self.classifier_tag(out_tag)
        out_pos = self.classifier_pos(out_pos)
        
        #calc loss
        loss_tag = loss_func(out_tag, target_tag, attention_mask, self.num_tag)
        loss_pos = loss_func(out_pos, target_pos, attention_mask, self.num_pos)
        loss = (loss_tag + loss_pos) / 2
#         each token in the sequence has `out_tag` or `out_pos` num of predictions 
#         so we need to softmax the predictions and take the max
#         return self.classifier_tag(out_tag), self.classifier_pos(out_pos), loss
        return out_tag, out_pos, loss

In [ ]:
#export
def loss_func(out, target, mask, num_labels, func=nn.CrossEntropyLoss()):
    '''loss func for NER tasks
        out is logit from the model. Shape (bs, seq_len, hidden_dim[num_labels])
        target is target from dataloader. Shape (bs, seq_len)
    '''
    #the mask tell us where non zero tokens are
    #the num_labels is used to tell us how many labels(le.classes_) are in the targ
    non_zero_tokens = mask.view(-1) == 1 # zeroed token_ids have a mask of 1
    ignore_index = func.ignore_index

#     if the token is not zero, select the corresponding target else set ignore_index
    cleaned_target = torch.where(non_zero_tokens.to(target.device), 
                                 target.view(-1), 
                                 torch.tensor(ignore_index).to(target.device).type_as(target)) #[bs*seq_len]

    cleaned_out = out.view(-1, num_labels) #[bs*seq_len, n++um_labels]

    loss = func(cleaned_out.to(target.device), cleaned_target.to(target.device))

#     loss = func(cleaned_out, target.view(-1))

    return loss